#### **Description**
Сервисы доставки еды уже давно перестали быть просто курьерами, которые привозят заказ. Индустрия e-grocery стремительно идет к аккумулированию и использованию больших данных, чтобы знать о своих пользователях больше и предоставлять более качественные и персонализированные услуги. Одним из шагов к такой персонализации может быть разработка модели, которая понимает привычки и нужды пользователя, и, к примеру, может угадать, что и когда пользователь захочет заказать в следующий раз.

Такая модель, будучи разработанной, может принести значительную ценность для клиента - сэкономить время при сборке корзины, помочь ничего не забыть в заказе, убрать необходимость планировать закупки и следить за заканчивающимися запасами продуктов.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt;
import lightgbm as lgb
from itertools import chain
from tqdm import tqdm
import numpy as np;
import time
tqdm.pandas()  # Активируем tqdm для pandas
from time import gmtime, strftime


In [3]:
# Загрузка данных
train = pd.read_csv('data/train.csv')
df_sample = pd.read_csv('data/sample_submission.csv')
df_sample=df_sample.drop(["target"],axis=1);

In [117]:
train.head(5)

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [118]:
train.describe()

,user_id,cart
count,3.123064e+06,3.123064e+06
mean,7.253373e+03,2.273235e+02
std,5.337838e+03,2.112867e+02
min,0.000000e+00,0.000000e+00
25%,2.884000e+03,4.200000e+01
50%,6.055000e+03,1.460000e+02
75%,1.117200e+04,3.990000e+02
max,1.999900e+04,8.800000e+02


In [119]:
# Отделим user_id от cart в sample, что бы было удобнее анализировать количество пользователей и количество категорий товаров.

#Оптмизируем операцию.
# Используем векторные операции pandas (str.split)
#это значительно ускоряет процесс за счет оптимизаторов pandas
#Избегаем медленных операций append в списки
#Избегаем многократного вызова split() для одних и тех же данных
#Сразу получаем numpy arrays через .values

split_data = df_sample['id'].str.split(';', expand=True) 
user_id = split_data[0].values
cart = split_data[1].values

# Создаем DataFrame
sample = pd.DataFrame({
    "user_id": user_id,
    "cart": cart
})

sample.head(3)

,user_id,cart
0,0,133
1,0,5
2,0,10


In [120]:
sample.describe()

,user_id,cart
count,790449,790449
unique,13036,858
top,380,57
freq,250,11336


In [121]:
df_new = train;

In [122]:
# Преобразуем колонку с датами в правильный формат
df_new['order_completed_at'] = pd.to_datetime(train['order_completed_at'])
df_new.head(3)

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198


In [123]:
df_new.describe()

,user_id,order_completed_at,cart
count,3.123064e+06,3123064,3.123064e+06
mean,7.253373e+03,2020-04-09 01:17:00.182836992,2.273235e+02
min,0.000000e+00,2015-03-22 09:25:46,0.000000e+00
25%,2.884000e+03,2020-02-03 06:03:43,4.200000e+01
50%,6.055000e+03,2020-05-19 06:35:20,1.460000e+02
75%,1.117200e+04,2020-07-14 04:50:22,3.990000e+02
max,1.999900e+04,2020-09-03 23:45:45,8.800000e+02
std,5.337838e+03,NaN,2.112867e+02


In [124]:
#Добавим фичу, среднее количество дней между покупками.
user_features = df_new.groupby('user_id').agg(total_orders=('order_completed_at', 'count'),            # Общее число заказов
    avg_days_between_orders=('order_completed_at', lambda x: x.diff().mean().days)  # Средний интервал мужде покупками
).reset_index()

In [125]:
user_features.head(3)

,user_id,total_orders,avg_days_between_orders
0,0,44,1
1,1,37,12
2,2,172,11


In [126]:
# Оценим популярность товара, т.е. частоту его появления! 
# Список всех категорий в каждом заказе
# Если итерируемый объект, то all_categories=list(chain.from_iterable(train['cart'])), иначе:
all_categories = df_new['cart'].tolist();
# Очениваем частоту появления 
category_features = pd.Series(all_categories).value_counts().reset_index()
#Добавим названия
category_features.columns = ['cart', 'cart_popular']  # Общая популярность

In [127]:
category_features.head(3)

,cart,cart_popular
0,57,108877
1,14,93957
2,61,91543


In [128]:
category_features.describe()

,cart,cart_popular
count,881.00000,881.000000
mean,440.00000,3544.908059
std,254.46709,10686.317664
min,0.00000,1.000000
25%,220.00000,7.000000
50%,440.00000,66.000000
75%,660.00000,1101.000000
max,880.00000,108877.000000


In [129]:
features = pd.merge(df_new, category_features, on='cart')

features.head(3)

,user_id,order_completed_at,cart,cart_popular
0,2,2015-03-22 09:25:46,399,13682
1,2,2015-03-22 09:25:46,14,93957
2,2,2015-03-22 09:25:46,198,17707


In [130]:
features.describe()

,user_id,order_completed_at,cart,cart_popular
count,3.123064e+06,3123064,3.123064e+06,3.123064e+06
mean,7.253373e+03,2020-04-09 01:17:00.182836992,2.273235e+02,3.572283e+04
min,0.000000e+00,2015-03-22 09:25:46,0.000000e+00,1.000000e+00
25%,2.884000e+03,2020-02-03 06:03:43,4.200000e+01,1.395800e+04
50%,6.055000e+03,2020-05-19 06:35:20,1.460000e+02,2.724900e+04
75%,1.117200e+04,2020-07-14 04:50:22,3.990000e+02,4.957100e+04
max,1.999900e+04,2020-09-03 23:45:45,8.800000e+02,1.088770e+05
std,5.337838e+03,NaN,2.112867e+02,2.892834e+04


In [131]:
# Считаем, сколько раз пользователь заказывал категорию
user_cat_features = df_new.groupby(['user_id', 'cart']).agg(
    cart_order_count=('order_completed_at', 'count'),         # Частота заказов по каждому user
    last_order_date=('order_completed_at', 'max'),            # Дата последнего заказа
    first_order_date=('order_completed_at', 'min')            # Дата первого заказа

).reset_index()

user_cat_features['diff_order_date']=(user_cat_features.last_order_date-user_cat_features.first_order_date).dt.days;
#.dt.days — сразу возвращает целое число дней (отбрасывая часы/минуты).

user_cat_features=user_cat_features.drop(["last_order_date", "first_order_date"], axis=1);

user_cat_features.head(3)

,user_id,cart,cart_order_count,diff_order_date
0,0,5,1,0
1,0,10,1,0
2,0,14,2,35


In [132]:
user_cat_features.describe()

,user_id,cart,cart_order_count,diff_order_date
count,1.117600e+06,1.117600e+06,1.117600e+06,1.117600e+06
mean,8.653122e+03,2.569703e+02,2.794438e+00,6.733846e+01
std,5.624315e+03,2.213021e+02,3.907522e+00,1.192000e+02
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,3.787000e+03,6.100000e+01,1.000000e+00,0.000000e+00
50%,7.990000e+03,1.970000e+02,1.000000e+00,0.000000e+00
75%,1.322400e+04,4.030000e+02,3.000000e+00,9.100000e+01
max,1.999900e+04,8.800000e+02,1.960000e+02,1.915000e+03


In [133]:
features = pd.merge(features, user_cat_features, on=['user_id','cart'], how="left")

features.head(3)

,user_id,order_completed_at,cart,cart_popular,cart_order_count,diff_order_date
0,2,2015-03-22 09:25:46,399,13682,1,0
1,2,2015-03-22 09:25:46,14,93957,1,0
2,2,2015-03-22 09:25:46,198,17707,2,1027


In [153]:
features.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.95])

,user_id,order_completed_at,cart,cart_popular,cart_order_count,diff_order_date,target
count,3.123064e+06,3123064,3.123064e+06,3.123064e+06,3.123064e+06,3.123064e+06,3.123064e+06
mean,7.253373e+03,2020-04-09 01:17:00.182836992,2.273235e+02,3.572283e+04,8.258405e+00,1.666952e+02,6.123141e-01
min,0.000000e+00,2015-03-22 09:25:46,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
10%,9.380000e+02,2019-10-26 21:07:12,1.700000e+01,4.914000e+03,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.884000e+03,2020-02-03 06:03:43,4.200000e+01,1.395800e+04,2.000000e+00,2.500000e+01,0.000000e+00
50%,6.055000e+03,2020-05-19 06:35:20,1.460000e+02,2.724900e+04,4.000000e+00,1.200000e+02,1.000000e+00
75%,1.117200e+04,2020-07-14 04:50:22,3.990000e+02,4.957100e+04,1.000000e+01,2.650000e+02,1.000000e+00
95%,1.749700e+04,2020-08-24 10:19:24,6.860000e+02,9.395700e+04,2.900000e+01,4.620000e+02,1.000000e+00
max,1.999900e+04,2020-09-03 23:45:45,8.800000e+02,1.088770e+05,1.960000e+02,1.915000e+03,1.000000e+00
std,5.337838e+03,NaN,2.112867e+02,2.892834e+04,1.108092e+01,1.809390e+02,4.872224e-01


In [135]:
#features = pd.merge(features, user_features, on=['user_id'], how="left")

#features.head(3)

In [ ]:
features["target"]=np.where((features['cart_popular'] > 5000) &         # Популярность
                             (features['cart_order_count'] >2)&          # Частота заказов по каждому user
                             (features['diff_order_date'] > 1), 1, 0)    # количество дней между первым и последним заказом

In [137]:
print(features['target'].value_counts())

target
1    1912296
0    1210768
Name: count, dtype: int64


In [138]:
features.describe()

,user_id,order_completed_at,cart,cart_popular,cart_order_count,diff_order_date,target
count,3.123064e+06,3123064,3.123064e+06,3.123064e+06,3.123064e+06,3.123064e+06,3.123064e+06
mean,7.253373e+03,2020-04-09 01:17:00.182836992,2.273235e+02,3.572283e+04,8.258405e+00,1.666952e+02,6.123141e-01
min,0.000000e+00,2015-03-22 09:25:46,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.884000e+03,2020-02-03 06:03:43,4.200000e+01,1.395800e+04,2.000000e+00,2.500000e+01,0.000000e+00
50%,6.055000e+03,2020-05-19 06:35:20,1.460000e+02,2.724900e+04,4.000000e+00,1.200000e+02,1.000000e+00
75%,1.117200e+04,2020-07-14 04:50:22,3.990000e+02,4.957100e+04,1.000000e+01,2.650000e+02,1.000000e+00
max,1.999900e+04,2020-09-03 23:45:45,8.800000e+02,1.088770e+05,1.960000e+02,1.915000e+03,1.000000e+00
std,5.337838e+03,NaN,2.112867e+02,2.892834e+04,1.108092e+01,1.809390e+02,4.872224e-01


In [139]:
features.head(3)

,user_id,order_completed_at,cart,cart_popular,cart_order_count,diff_order_date,target
0,2,2015-03-22 09:25:46,399,13682,1,0,0
1,2,2015-03-22 09:25:46,14,93957,1,0,0
2,2,2015-03-22 09:25:46,198,17707,2,1027,0


In [140]:
target=features['target'];
features_2=features[["user_id", "cart"]];

In [141]:
features_2.head(3)

,user_id,cart
0,2,399
1,2,14
2,2,198


In [142]:
from sklearn.model_selection import train_test_split

In [143]:
# Разделение user_id на train/valid
X_train, X_test, y_train, y_test = train_test_split(
    features_2, target,
    test_size=0.2,
    random_state=42,
)

In [144]:
print("y_train.shape = ", y_train.shape)
print("X_train.shape = ", X_train.shape)

print("y_test.shape = ", y_test.shape)
print("X_test.shape = ", X_test.shape)

y_train.shape =  (2498451,)
X_train.shape =  (2498451, 2)
y_test.shape =  (624613,)
X_test.shape =  (624613, 2)


#### Получим опорное решение.

In [ ]:
%%time

from sklearn import ensemble
import pickle

model = (ensemble.RandomForestClassifier(n_estimators=100, random_state=3,verbose=True,\
                                         n_jobs=-1,oob_score=False,max_features='sqrt'));
model.fit(X_train, y_train)

#filename = 'model_one'+strftime("%H-%M-%S")+'.sav'
#pickle.dump(model, open(filename, 'wb'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.7min


CPU times: total: 28min 39s
Wall time: 3min 3s


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  3.0min finished


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=3,
                       verbose=True)

In [146]:
score = model.score(X_test, y_test)
print(f"Accuracy: {score:.2f}")

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.3s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   12.8s


Accuracy: 0.95


[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:   14.2s finished


In [ ]:
# Переберем несколько моделей.

#Пуская 
#n_estimators_list = [10, 50, 100, 200, 300]
#max_depth_list = [None, 5, 10, 15]
#results = []

# Перебираем модели с tqdm
#for n_estimators in tqdm(n_estimators_list, desc="Number of Trees"):
 #   for max_depth in tqdm(max_depth_list, desc="Max Depth", leave=False):
  #      model = ensemble.RandomForestClassifier(
   #         n_estimators=n_estimators,
    #        max_depth=max_depth,
     #       random_state=42
      #  )
       # model.fit(X_train, y_train)
        #score = model.score(X_test, y_test)
        #results.append({
         #   'n_estimators': n_estimators,
          #  'max_depth': max_depth,
           # 'accuracy': score
        #})

# Выводим результаты
#for res in results:
 #   print(f"Trees: {res['n_estimators']}, Depth: {res['max_depth']}, Accuracy: {res['accuracy']:.4f}")

- Trees: 10, Depth: None, Accuracy: 0.9340
- Trees: 10, Depth: 5, Accuracy: 0.7168
- Trees: 10, Depth: 10, Accuracy: 0.7497
- Trees: 10, Depth: 15, Accuracy: 0.7723
- Trees: 50, Depth: None, Accuracy: 0.9425
- Trees: 50, Depth: 5, Accuracy: 0.7189
- Trees: 50, Depth: 10, Accuracy: 0.7498
- Trees: 50, Depth: 15, Accuracy: 0.7726
- Trees: 100, Depth: None, Accuracy: 0.9433
- Trees: 100, Depth: 5, Accuracy: 0.7189
- Trees: 100, Depth: 10, Accuracy: 0.7499
- Trees: 100, Depth: 15, Accuracy: 0.7729
- Trees: 200, Depth: None, Accuracy: 0.9433
- Trees: 200, Depth: 5, Accuracy: 0.7190
- Trees: 200, Depth: 10, Accuracy: 0.7499
- Trees: 200, Depth: 15, Accuracy: 0.7728
- Trees: 300, Depth: None, Accuracy: 0.9433
- Trees: 300, Depth: 5, Accuracy: 0.7179
- Trees: 300, Depth: 10, Accuracy: 0.7495
- Trees: 300, Depth: 15, Accuracy: 0.7727

Подбирем параметры "получше".Внимание, это самостоятельная работа!!!
- Оптимальный баланс между скоростью и качеством — **RandomizedSearchCV**. Он проверяет случайные комбинации параметров, а не все возможные (как GridSearchCV).
- Использование HalvingRandomSearchCV (еще быстрее!), но это если scikit-learn версии 0.24+.**HalvingRandomSearchCV** — отсеивает плохие комбинации на ранних этапах, экономя время.

In [148]:
# Вариант с RandomizedSearchCV

'''from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Параметры для поиска
param_dist = {
    'n_estimators': randint(20, 300),       # Количество деревьев
    'max_depth': randint(2, 15),            # Глубина дерева
    'min_samples_split': randint(2, 10),    # Минимальное число образцов для разделения
    'min_samples_leaf': randint(1, 6),      # Минимальное число образцов в листе
    'max_features': ['sqrt', 'log2', None]  # Количество признаков для разделения
}

model = RandomForestClassifier(random_state=3, n_jobs=-1)

# Запуск RandomizedSearchCV
search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=30,  # Количество случайных комбинаций (чем больше, тем дольше)
    cv=3,       # Количество фолдов кросс-валидации
    scoring='accuracy',
    random_state=3,
    n_jobs=-1
)

search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", search.best_params_)
print("Лучшая точность:", search.best_score_)
'''

'from sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom scipy.stats import randint\n\n# Параметры для поиска\nparam_dist = {\n    \'n_estimators\': randint(20, 300),       # Количество деревьев\n    \'max_depth\': randint(2, 15),            # Глубина дерева\n    \'min_samples_split\': randint(2, 10),    # Минимальное число образцов для разделения\n    \'min_samples_leaf\': randint(1, 6),      # Минимальное число образцов в листе\n    \'max_features\': [\'sqrt\', \'log2\', None]  # Количество признаков для разделения\n}\n\nmodel = RandomForestClassifier(random_state=3, n_jobs=-1)\n\n# Запуск RandomizedSearchCV\nsearch = RandomizedSearchCV(\n    model,\n    param_distributions=param_dist,\n    n_iter=30,  # Количество случайных комбинаций (чем больше, тем дольше)\n    cv=3,       # Количество фолдов кросс-валидации\n    scoring=\'accuracy\',\n    random_state=3,\n    n_jobs=-1\n)\n\nsearch.fit(X_train, y_train)\n\n# Лучшие параме

### Сформируем predict и submisson

In [150]:
from time import gmtime, strftime
tm=strftime("%H-%M-%S");

sab=model.predict(sample);

sabm=pd.DataFrame();
sabm["target"]=sab;

s=sample["user_id"]+";"+sample["cart"];
s=pd.DataFrame(s)

s.columns=["id"];
s["target"]=sabm;

fil="my_submission "+str(tm)+str('.csv');
s.to_csv(str(fil), index=False)# Записываем результат в файл
print("Your submission was successfully saved!")# Сообщение, если все хорошо

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    2.2s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    2.5s finished


Your submission was successfully saved!
